In [1]:
import sys
import os

# Pobieramy ścieżkę do katalogu głównego projektu (dwa poziomy wyżej: ../..)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Dodajemy ją do ścieżek Pythona, jeśli jej tam nie ma
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import json

from app.database.database import Database
from app.database.sentence_transformer_embedder import SentenceTransformerEmbedder
from app.handler.defaulthandler import PromptHandler
from app.interface.abstractmodel import AbstractModel
from app.prompts_classification.fact_or_generative_classifier import FactOrGenerativeClassifier

with open('programming_questions.json', 'r') as f:
	data = json.load(f)

data

/home/xxxdkgxxx/PycharmProjects/BHL-solution/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'original_question': 'Czym jest rekurencja?',
  'cached_answer': 'Rekurencja to technika programistyczna, w której funkcja wywołuje samą siebie w celu rozwiązania problemu. Kluczowym elementem jest warunek bazowy, który przerywa ciąg wywołań, zapobiegając nieskończonej pętli i przepełnieniu stosu.',
  'paraphrases': ['Wyjaśnij pojęcie funkcji rekurencyjnej.',
   'Na czym polega wywoływanie funkcji przez samą siebie?',
   'Jak działa algorytm rekurencyjny?'],
  'hard_negatives': ['Jak naprawić błąd RecursionError?',
   'Czym różni się iteracja od rekurencji?']},
 {'original_question': 'Jak działa Garbage Collector w Javie?',
  'cached_answer': 'Garbage Collector (GC) to proces automatycznego zarządzania pamięcią, który identyfikuje i usuwa obiekty, do których nie ma już żadnych referencji w programie. Działa w tle, zwalniając zasoby na stercie (heap), co zapobiega wyciekom pamięci.',
  'paraphrases': ['Wyjaśnij mechanizm odśmiecania pamięci w JVM.',
   'Kiedy usuwane są nieużywane obi

In [3]:
class MockLLM(AbstractModel):

	def generate_answer(self, prompt: str) -> str:
		return "Answer"


db = Database(SentenceTransformerEmbedder())

handler = PromptHandler(db, MockLLM(), FactOrGenerativeClassifier())

for question in data:
	print(f"Question: {question['original_question']}")
	db.insert(question["original_question"], question["cached_answer"])

Question: Czym jest rekurencja?
Question: Jak działa Garbage Collector w Javie?
Question: Co to jest REST API?
Question: Jaka jest różnica między listą a krotką w Pythonie?
Question: Do czego służy polecenie git commit?
Question: Czym jest SQL Injection?
Question: Co oznacza akronim SOLID?
Question: Czym jest Docker?
Question: Co to jest złożoność obliczeniowa Big O?
Question: Na czym polega dziedziczenie w OOP?
Question: Czym są Promises w JavaScript?
Question: Co to jest klucz główny (Primary Key)?
Question: Czym jest framework React?
Question: Co to jest CI/CD?
Question: Czym jest typowanie statyczne?
Question: Co to jest CSS Grid?
Question: Wyjaśnij wzorzec Singleton.
Question: Czym jest funkcja lambda?
Question: Co to jest JSON?
Question: Czym jest wskaźnik (pointer) w C++?


In [4]:
expected_hits = 0
got_hits = 0
expected_misses = 0
got_misses = 0

for question in data:
    for positive_similar in question["paraphrases"]:
        print(f"Checking paraphrase: {positive_similar}")

        _, is_cached, _ = handler.generate_answer(positive_similar, False)

        if is_cached:
            got_hits += 1
        expected_hits += 1

    for negative in question["hard_negatives"]:
        print(f"Checking negative: {negative}")

        _, is_cached, _ = handler.generate_answer(negative, False)

        if not is_cached:
            got_misses += 1
        expected_misses += 1

    print("-" * 40)
    print(f"Processed question group: {question['original_question']}")
    print(f"  > Hits (Cache Hits):   {got_hits}/{expected_hits}")
    print(f"  > Misses (Cache Miss): {got_misses}/{expected_misses}")
    print("-" * 40 + "\n")

print("=== FINAL SUMMARY ===")
print(f"Total Accuracy (Hits):   {got_hits}/{expected_hits} ({(got_hits/expected_hits)*100:.1f}%)")
print(f"Total Accuracy (Misses): {got_misses}/{expected_misses} ({(got_misses/expected_misses)*100:.1f}%)")

Checking paraphrase: Wyjaśnij pojęcie funkcji rekurencyjnej.
Checking paraphrase: Na czym polega wywoływanie funkcji przez samą siebie?
{'math': 4.348517904873006e-05, 'bio': 1.790138594515156e-05, 'code': 1.904554846987594e-05}
Checking paraphrase: Jak działa algorytm rekurencyjny?
{'math': 0.0008533305954188108, 'bio': 5.040765609010123e-05, 'code': 6.453808600781485e-05}
Checking negative: Jak naprawić błąd RecursionError?
{'math': 0.00043839626596309245, 'bio': 2.2312047804007307e-05, 'code': 5.8865851315204054e-05}
Checking negative: Czym różni się iteracja od rekurencji?
----------------------------------------
Processed question group: Czym jest rekurencja?
  > Hits (Cache Hits):   1/3
  > Misses (Cache Miss): 1/2
----------------------------------------

Checking paraphrase: Wyjaśnij mechanizm odśmiecania pamięci w JVM.
{'math': 0.0012703537940979004, 'bio': 5.846612839377485e-05, 'code': 9.869701898423955e-05}
Checking paraphrase: Kiedy usuwane są nieużywane obiekty w Javie?
{

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 5.60 GiB of which 4.00 MiB is free. Process 31201 has 272.00 MiB memory in use. Including non-PyTorch memory, this process has 5.33 GiB memory in use. Of the allocated memory 4.83 GiB is allocated by PyTorch, and 400.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)